In [3]:
# !pip install -qqq -U git+https://github.com/huggingface/transformers.git
# !pip install -qqq -U git+https://github.com/huggingface/peft.git
# !pip install -qqq -U git+https://github.com/huggingface/accelerate.git
# !pip install -qqq bitsandbytes
# !pip install scipy
# !pip install nltk
# !pip install google-cloud-vision
# !pip install datasets
# !pip install evaluate

In [4]:
import io
import os
from tqdm import tqdm
import concurrent.futures
from google.cloud import vision_v1
from google.cloud.vision_v1 import types

# path for the JSON key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'Your vision key.json'

import json
import re
import time
import zipfile
import shutil
from pprint import pprint
from datasets import load_dataset
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datetime import datetime
from datasets import load_dataset
import numpy as np
import nltk
import evaluate
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
base_model_id = "HuggingFaceH4/zephyr-7b-alpha"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=4096,
    padding=True,
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token
ft_model = PeftModel.from_pretrained(base_model, "your generated Fine-Tuned Checkpoint")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [8]:
def model_output_batch(ocr_texts, ft_model, tokenizer, batch_size=10):
    """
    Processes a batch of OCR texts and extracts information using the finetuned LLM.

    :param ocr_texts: List of OCR text strings.
    :param ft_model: The finetuned language model.
    :param tokenizer: The tokenizer for the language model.
    :param batch_size: Number of texts to process in a single batch.
    :return: List of processed outputs.
    """
    # Ensure the model is in evaluation mode and on the GPU
    ft_model.eval()
    ft_model.to('cuda')

    all_outputs = []

    # Use tqdm for the progress bar
    for i in tqdm(range(0, len(ocr_texts), batch_size), desc="Processing batches", unit="batch"):
        # Prepare the batch
        batch_texts = ocr_texts[i:i + batch_size]

        # Constructing eval_prompt for each text in the batch
        # Inferencing and Training Prompt should be SAME !!!
        
        eval_prompts = [
            f"""Extract specific data from the provided OCR text of a marriage record. Identify and return the following details in a structured JSON format: Application number, date of marriage ( month,day, year), place of marriage, birth details (month,day, year, place) of both spouses, their county, gender, given names, residence, state, and surnames. Only show the extracted information. Don't show the OCR Text again. Also don't show the pattern which you are asked to follow. Concentrate on the data asked to extract . The output must strictly adhere to the given details and key names, starting with an opening curly brace '{' and ending with a closing curly brace '}'. Ensure that each key-value pair contains only accurate and relevant information from the OCR text. Avoid including any extraneous or irrelevant data that does not correspond directly to the specified keys. The structure of the output is as follows:

Application: The marriage Application number.
Spouse1_Given: The first name of the first spouse.
Spouse1_Surname: The last name of the first spouse.
Spouse1_Birth_Day: The birth day of the first spouse.
Spouse1_Birth_Month: The birth month of the first spouse.
Spouse1_Birth_Year: The birth year of the first spouse.
Spouse1_Residence: The residence of the first spouse.
Spouse1_County: The county of the first spouse.
Spouse1_State: The state of the first spouse.
Spouse1_Birthplace: The birthplace of the first spouse.
Spouse1_Gender: The gender of the first spouse.
Spouse2_Given: The first name of the second spouse.
Spouse2_Surname: The last name of the second spouse.
Spouse2_Maiden: The maiden name of the second spouse, if applicable.
Spouse2_Birth_Day: The birth day of the second spouse.
Spouse2_Birth_Month: The birth month of the second spouse.
Spouse2_Birth_Year: The birth year of the second spouse.
Spouse2_Residence: The residence of the second spouse.
Spouse2_County: The county of the second spouse.
Spouse2_State: The state of the second spouse.
Spouse2_Birthplace: The birthplace of the second spouse.
Spouse2_Gender: The gender of the second spouse.
Marriage_Day: The day of the marriage.
Marriage_Month: The month of the marriage.
Marriage_Year: The year of the marriage.
Marriage_Place: The location of the marriage.

    ### OCR Text:
    {text}
    """ for text in batch_texts]

        # Tokenizing the batch of prompts
        batch_inputs = tokenizer(eval_prompts, padding=True, return_tensors="pt").to("cuda")

        # Perform inference in a memory-efficient way
        with torch.no_grad():
            batch_outputs = ft_model.generate(**batch_inputs, max_new_tokens=600)
            decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in batch_outputs]

        all_outputs.extend(decoded_outputs)

    return all_outputs


In [9]:
def process_images_and_infer_parallel(directory_path, ft_model, tokenizer, max_workers=8):
    """
    Parallelized function to process images, perform OCR, and infer using the finetuned LLM.

    :param directory_path: Path to the directory containing images.
    :param max_workers: Maximum number of parallel threads for image processing.
    :return: List of dictionaries containing the results for each image.
    """
    def process_image(file_path):
        client = vision_v1.ImageAnnotatorClient()
        image_name, _ = os.path.splitext(os.path.basename(file_path))

        with io.open(file_path, 'rb') as image_file:
            content = image_file.read()

        image = vision_v1.Image(content=content)
        response = client.document_text_detection(image=image)

        confidences = []
        bounding_boxes = []
        ocr_data = ""

        if response.text_annotations:
            ocr_data = response.text_annotations[0].description.replace('\n', ' ')
            ocr_data = json.dumps(ocr_data)
            for page in response.full_text_annotation.pages:
                for block in page.blocks:
                    for paragraph in block.paragraphs:
                        for word in paragraph.words:
                            word_text = ''.join([symbol.text for symbol in word.symbols])
                            confidences.append((word_text, word.confidence))
                            vertices = [(v.x, v.y) for v in word.bounding_box.vertices]
                            bounding_boxes.append((word_text, vertices))

        return image_name, ocr_data, confidences, bounding_boxes

    image_files = [f for f in os.listdir(directory_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.jfif'))]

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_image = {executor.submit(process_image, os.path.join(directory_path, filename)): filename for filename in image_files}

        ocr_texts = []
        image_details = []

        for future in tqdm(concurrent.futures.as_completed(future_to_image), total=len(future_to_image), desc="Processing Images"):
            image_name, ocr_data, confidences, bounding_boxes = future.result()
            tqdm.write(f"Processed OCR for image: {image_name}")
            image_details.append({
                'Image': image_name,
                'OCR': ocr_data,
                'Confidence Score': confidences,
                'Bounding Boxes': bounding_boxes
            })
            ocr_texts.append(ocr_data)

    # Process OCR texts with the language model
    tqdm.write("Inferring from OCR data using the model...")
    inferencing_results = model_output_batch(ocr_texts, ft_model, tokenizer)

    # Attach the inferencing results to image_details
    for detail, inference_output in zip(image_details, inferencing_results):
        detail['Model_Output'] = inference_output
        tqdm.write(f"Inferred model output for image: {detail['Image']}")

    return image_details

In [10]:
def extract_json_outputs_parallel(final_results, max_workers=8):
    """
    Parallelized function to extract and modify JSON outputs from the results.

    :param final_results: List of dictionaries, each being an output from the previous function.
    :param max_workers: Number of parallel threads.
    :return: List of modified JSON outputs.
    """
    def replace_keys(data):
        modified_data = {}
        for key, value in data.items():
            modified_key_orig = key + '_orig'
            modified_data[modified_key_orig] = value
            modified_key_hwr = key + '_hwr'
            modified_data[modified_key_hwr] = value
        return modified_data

    def process_item(item):
        inference_text = item['Model_Output']
        
        # Use the provided regular expression pattern to find the JSON string
        #pattern = r'\{\s*"Application number": "[\s\S]+?Spouse2_surname": "[^\"]*"\s*\}'
        #pattern = r'\{\s*"?Application\s*number"?\s*:\s*"[^"]*?"(?:\s*,\s*"[^"]*?"\s*:\s*"(?:\\.|[^"\\])*")*?\s*,\s*"?Spouse2_surname"?\s*:\s*"(?:\\.|[^"\\])*"\s*\}'

        pattern = (r'\{\s*"?Application\s*number"?\s*:\s*"[^"]*?"'
                    r'(?:\s*,\s*"(?:\\.|[^"\\])*?"\s*:\s*"(?:\\.|[^"\\])*")*?'
                    r'\s*,\s*"?Spouse2_surname"?\s*:\s*"(?:\\.|[^"\\])*?"\s*\}'
                )

        match = re.search(pattern, inference_text)
    
        # If no match is found, return None
        if not match:
            return None
    
        json_string = match.group()
    
        try:
            json_data = json.loads(json_string)
            json_data = replace_keys(json_data)
            json_data['Image'] = item['Image']
            return json_data
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for image {item['Image']}: {e}")
            return None

    json_outputs = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_item, item) for item in final_results]

        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                json_outputs.append(result)

    return json_outputs


In [15]:
start_time = time.time()
print(f"Start time: {start_time} seconds \n")

directory_path = "your input image directory path"
final_results = process_images_and_infer_parallel(directory_path, ft_model, tokenizer)
llm_json_output = extract_json_outputs_parallel(final_results)
#print(llm_json_output)

end_time = time.time()
print(f"End time: {end_time} seconds\n")

# Calculate and log the elapsed time
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time} seconds")
print("Execution Completed!")

Start time: 1705436625.8016274 seconds 



Processed OCR for image: 2008225268_01


Processed OCR for image: 2008225243_01


Processing Images:   0%|          | 3/3053 [00:02<39:09,  1.30it/s]

Processed OCR for image: 2008225280_01


...

Processed OCR for image: 2009076291_01
Processed OCR for image: 2009076162_01
Processed OCR for image: 2009076316_01
Processed OCR for image: 2009076303_01
Processed OCR for image: 2009076358_01
Processed OCR for image: 2009076378_01
Processed OCR for image: 2009076393_01
Processed OCR for image: 2009076564_01
Processed OCR for image: 2009076483_01
Processed OCR for image: 2009076675_01
Processed OCR for image: 2009076746_01


Processing Images: 100%|██████████| 3053/3053 [14:17<00:00,  3.56it/s]


Processed OCR for image: 2009076824_01
Processed OCR for image: 2009076960_01
Inferring from OCR data using the model...


Processing batches:   0%|          | 0/306 [00:00<?, ?batch/s]/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1553: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing batches: 100%|██████████| 306/306 [8:18:09<00:00, 97.68s/batch]


Inferred model output for image: 2008225268_01
Inferred model output for image: 2008225243_01
Inferred model output for image: 2008225280_01
Inferred model output for image: 2008225261_01
Inferred model output for image: 2008225173_01
Inferred model output for image: 2008225114_01
Inferred model output for image: 2008225077_01
Inferred model output for image: 2008225373_01
Inferred model output for image: 2008225113_01
Inferred model output for image: 2008225543_01
Inferred model output for image: 2008225674_01
Inferred model output for image: 2008226267_01
Inferred model output for image: 2008225988_01
Inferred model output for image: 2008225618_01
Inferred model output for image: 2008226090_01
Inferred model output for image: 2008226376_01
Inferred model output for image: 2008226420_01
Inferred model output for image: 2008226444_01
Inferred model output for image: 2008226446_01
Inferred model output for image: 2008226447_01
Inferred model output for image: 2008226448_01
Inferred mode